# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification


### Introduction

The project folder has the following structure:

* In the main directory you have this notebook, `cnn_from_scratch.ipynb`, that contains the instruction and some questions you will have to answer. Follow this notebook and complete the required sections in order.

* In the `src/` directory you have several source files. As instructed in this notebook, you will open and complete those files, then come back to this notebook to execute some tests that will verify what you have done. While these tests don't guarantee that your work is bug-free, they will help you finding the most obvious problems so you will be able to proceed to the next step with confidence.

* Sometimes you will need to restart the notebook. If you do so, remember to execute also the cells containing the code you have already completed starting from the top, before you move on.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.

### Designing and training a CNN from scratch

In this notebook, you will create a CNN that classifies landmarks.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 50%.

Although 50% may seem low at first glance, it seems more reasonable after realizing how difficult of a problem this is. Many times, an image that is taken at a landmark captures a fairly mundane image of an animal or plant, like in the following picture.

<img src="static_images/train/00.Haleakala_National_Park/084c2aa50d0a9249.jpg" alt="Bird in Haleakalā National Park" style="width: 400px;"/>

Just by looking at that image alone, would you have been able to guess that it was taken at the Haleakalā National Park in Hawaii?

An accuracy of 50% is significantly better than random guessing, which would provide an accuracy of just 2% (100% / 50 classes). In Step 2 of this notebook, you will have the opportunity to greatly improve accuracy by using transfer learning to create a CNN.

Experiment with different architectures, hyperparameters, training strategies, and trust your intuition.  And, of course, have fun!

---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 0: Setting up

The following cells make sure that your environment is setup correctly, download the data if you don't have it already, and also check that your GPU is available and ready to go. You have to execute them every time you restart your notebook.

In [1]:
import os
import shutil

zip_file_path = os.path.join(os.getcwd(), '/content/starter_kit.zip')

unzip_dir = os.path.join(os.getcwd(), 'unzipped_files')

os.makedirs(unzip_dir, exist_ok = True)

shutil.unpack_archive(zip_file_path, unzip_dir , 'zip' )

print(f"Files unzipped to :{unzip_dir}")

Files unzipped to :/content/unzipped_files


In [2]:
# Install requirements
!pip install -r /content/unzipped_files/starter_kit/requirements.txt | grep -v "already satisfied"

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
import sys

sys.path.append('/content/unzipped_files/starter_kit/src/data.py')
sys.path.append('/content/unzipped_files/starter_kit/src/helpers.py')
sys.path.append('/content/unzipped_files/starter_kit/src/model.py')
sys.path.append('/content/unzipped_files/starter_kit/src/optimization.py')
sys.path.append('/content/unzipped_files/starter_kit/src/predictor.py')
sys.path.append('/content/unzipped_files/starter_kit/src/train.py')
sys.path.append('/content/unzipped_files/starter_kit/src/transfer.py')

Important! After you have ran pip install, please restart the kernel using the **Kernel** menu

In [4]:
from unzipped_files.starter_kit.src.helpers import setup_env

# If running locally, this will download dataset (make sure you have at
# least 2 Gb of space on your hard drive)
setup_env()

GPU available
done


Computing std: 100%|███████████████████████| 6246/6246 [01:00<00:00, 102.91it/s]


---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 1: Data

In this and the following steps we are going to complete some code, and then execute some tests to make sure the code works as intended.

Open the file `src/data.py`. It contains a function called `get_data_loaders`. Read the function and complete all the parts marked by `YOUR CODE HERE`. Once you have finished, test that your implementation is correct by executing the following cell (see below for what to do if a test fails):

In [5]:
!pytest -vv unzipped_files/starter_kit/src/data.py -k data_loaders

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 4 items / 1 deselected / 3 selected                                                      

unzipped_files/starter_kit/src/data.py::test_data_loaders_keys PASSED                        [ 33%]
unzipped_files/starter_kit/src/data.py::test_data_loaders_output_type PASSED                 [ 66%]
unzipped_files/starter_kit/src/data.py::test_data_loaders_output_shape PASSED                [100%]

================================= 3 passed, 1 deselected in 6.43s ==================================


You should see something like:
```
src/data.py::test_data_loaders_keys PASSED                               [ 33%]
src/data.py::test_data_loaders_output_type PASSED                        [ 66%]
src/data.py::test_data_loaders_output_shape PASSED                       [100%]

======================= 3 passed, 1 deselected in 1.81s ========================
```
If all the tests are `PASSED`, you can move to the next section.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> **What to do if tests fail**
When a test fails, `pytest` will mark it as `FAILED` as opposed to `PASSED`, and will print a lot of useful output, including a message that should tell you what the problem is. For example, this is the output of a failed test:
> ```
>    def test_data_loaders_keys(data_loaders):
>    
>       assert set(data_loaders.keys()) == {"train", "valid", "test"}
E       AssertionError: assert {'tes', 'train', 'valid'} == {'test', 'train', 'valid'}
E         Extra items in the left set:
E         'tes'
E         Full diff:
E         - {'test', 'train', 'valid'}
E         + {'tes', 'train', 'valid'}
E         ?                          +++++++
>
> src/data.py:171: AssertionError
-------------- Captured stdout setup ----------------------------------------------
Reusing cached mean and std for landmark_images
Dataset mean: tensor([0.4638, 0.4725, 0.4687]), std: tensor([0.2699, 0.2706, 0.3018])
=========== short test summary info ===============================================
FAILED src/data.py::test_data_loaders_keys - AssertionError: The keys of the data_loaders dictionary should be train, valid and test
> ```
> In the `short test summary info` you can see a short description of the problem. In this case, the dictionary we are returning has the wrong keys. Going above a little, you can see that the test expects `{'test', 'train', 'valid'}` while we are returning `{'tes', 'train', 'valid'}` (there is a missing `t`). So we can go back to our function, fix that problem and test again.
>
> In other cases, you might get an error like:
> ```
    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]):
        if self.padding_mode != 'zeros':
            return F.conv2d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, bias, self.stride,
                            _pair(0), self.dilation, self.groups)
        return F.conv2d(input, weight, bias, self.stride,
>                       self.padding, self.dilation, self.groups)
E       RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same
>
> ../../../../miniconda3/envs/udacity_starter/lib/python3.7/site-packages/torch/nn/modules/conv.py:440: RuntimeError
> ```
> Looking at the stack trace you should be able to understand what it is going on. In this case, we forgot to add a `.cuda()` to some tensor. For example, the model is on the GPU, but the data aren't.

<img src="static_images/icons/noun-question-mark-869751.png" alt="?" style="width:25px"/> **Question:** Describe your chosen procedure for preprocessing the data.
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?

## answers:

#### 1. Describe your chosen procedure for preprocessing the data:

I have developed a preprocessing pipeline that includes several essential steps to prepare the image data for training, validation, and testing. This pipeline involves resizing, cropping, normalization, and various data augmentation techniques.

#### 2. How does your code resize the images (by cropping, stretching, etc)? What size did you pick for the input tensor, and why?
To resize the images, I use the transforms.Resize(256) function to scale them to a resolution of 256x256 pixels. For the final input tensor size, I apply cropping. During training, I extract a random crop of 224x224 pixels using transforms.RandomCrop(224), while for validation and testing, I use transforms.CenterCrop(224) to get a center crop of the same size. The choice of 224x224 pixels for the input tensor is based on its compatibility with well-known CNN architectures like VGG16, ResNet, and InceptionV3, balancing computational efficiency with the ability to capture essential image features.

#### 3. Did you decide to augment the dataset? If so, how (through translations, flips, rotations, etc)? If not, why not?
Yes, I decided to augment the dataset to enhance its diversity and improve the model's robustness. I applied several augmentation techniques, including random horizontal flipping, random rotation, color jitter, random affine and perspective transformations, random grayscale conversion, and random erasing. These transformations introduce variations in the training data, making the model more resilient to different image presentations.

### Visualize a Batch of Training Data

Go back to `src/data.py` and complete the function `visualize_one_batch` in all places with the `YOUR CODE HERE` marker. After you're done, execute the following cell and make sure the test `src/data.py::test_visualize_one_batch` is `PASSED`:

In [6]:
!pytest -vv unzipped_files/starter_kit/src/data.py -k visualize_one_batch

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 4 items / 3 deselected / 1 selected                                                      

unzipped_files/starter_kit/src/data.py::test_visualize_one_batch PASSED                      [100%]

================================= 1 passed, 3 deselected in 3.23s ==================================


We can now use the code we just completed to get a batch of images from your train data loader and look at them.

Visualizing the output of your data loader is a great way to ensure that your data loading and preprocessing (including transforms such as rotations, translations, color transforms...) are working as expected.

In [7]:
!pip install pytest

Reusing cached mean and std
Dataset mean: tensor([0.4638, 0.4725, 0.4687]), std: tensor([0.2697, 0.2706, 0.3017])
Reusing cached mean and std


In [8]:
%matplotlib inline
from unzipped_files.starter_kit.src.data import visualize_one_batch, get_data_loaders

# use get_data_loaders to get the data_loaders dictionary. Use a batch_size
# of 5, a validation size of 0.01 and num_workers=-1 (all CPUs)
data_loaders = get_data_loaders(batch_size=5, valid_size=0.01, num_workers=1)

visualize_one_batch(data_loaders)

---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 2: Define model

Open `src/model.py` and complete the `MyModel` class filling in all the `YOUR CODE HERE` sections. After you're done, execute the following test and make sure it passes:

In [9]:
!pytest -vv unzipped_files/starter_kit/src/model.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 1 item                                                                                   

unzipped_files/starter_kit/src/model.py::test_model_construction PASSED                      [100%]

======================================== 1 passed in 5.59s =========================================


<img src="static_images/icons/noun-question-mark-869751.png" alt="?" style="width:25px"/> **Question**: Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

<img src="static_images/icons/noun-answer-3361020.png" alt=">" style="width:25px"/>

## Answer:

## Building a Powerful Image Classifier

To create a robust Convolutional Neural Network (CNN) for classifying images into 1000 categories, I carefully designed its architecture. I studied successful models like VGG, ResNet, and InceptionV3 to understand their strengths and incorporate key elements.

The core of the network consisted of five building blocks, each growing in complexity. The initial layers learned simple features, while deeper layers captured increasingly intricate details. To prevent the network from becoming overwhelmed, I included techniques like batch normalization and dropout.

I used consistent filter sizes and padding to maintain image dimensions across layers, simplifying the architecture. Max pooling gradually reduced image size, focusing on essential features.

Finally, the classifier section transformed the processed image into a probability distribution across 1000 categories. This design allows for flexibility, making it adaptable to different image datasets and classification tasks.


---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 3: define loss and optimizer

Open `src/optimization.py` and complete the `get_loss` function, then execute the test and make sure it passes:

In [10]:
!pytest -vv unzipped_files/starter_kit/src/optimization.py -k get_loss

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 7 items / 6 deselected / 1 selected                                                      

unzipped_files/starter_kit/src/optimization.py::test_get_loss PASSED                         [100%]

================================= 1 passed, 6 deselected in 4.15s ==================================


Then, in the same file, complete the `get_optimizer` function then execute its tests, and make sure they all pass:

In [11]:
!pytest -vv unzipped_files/starter_kit/src/optimization.py -k get_optimizer

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 7 items / 1 deselected / 6 selected                                                      

unzipped_files/starter_kit/src/optimization.py::test_get_optimizer_type PASSED               [ 16%]
unzipped_files/starter_kit/src/optimization.py::test_get_optimizer_is_linked_with_model PASSED [ 33%]
unzipped_files/starter_kit/src/optimization.py::test_get_optimizer_returns_adam PASSED       [ 50%]
unzipped_files/starter_kit/src/optimization.py::test_get_optimizer_sets_learning_rate PASSED [ 66%]
unzipped_files/starter_kit/src/optimization.py::test_get_optimizer_sets_momentum PASSED      [ 83%]
unzipped_files/starter_kit/src/optimization.py::test_get_optimizer_sets_weight_decay PASSED  [100%]

================================= 

---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 4: Train and Validate the Model

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Testing ML code is notoriously difficult. The tests in this section merely exercise the functions you are completing, so it will help you catching glaring problems but it won't guarantee that your training code is bug-free. If you see that your loss is not decreasing, for example, that's a sign of a bug or of a flawed model design. Use your judgement.

Open `src/train.py` and complete the `train_one_epoch` function, then run the tests:

In [12]:
!pytest -vv unzipped_files/starter_kit/src/train.py -k train_one_epoch

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 4 items / 3 deselected / 1 selected                                                      

unzipped_files/starter_kit/src/train.py::test_train_one_epoch PASSED                         [100%]

================================= 1 passed, 3 deselected in 10.91s =================================


Now complete the `valid` function, then run the tests:

In [13]:
!pytest -vv unzipped_files/starter_kit/src/train.py -k valid_one_epoch

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 4 items / 3 deselected / 1 selected                                                      

unzipped_files/starter_kit/src/train.py::test_valid_one_epoch PASSED                         [100%]

================================= 1 passed, 3 deselected in 6.13s ==================================


Now complete the `optimize` function, then run the tests:

In [14]:
!pytest -vv unzipped_files/starter_kit/src/train.py -k optimize

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 4 items / 3 deselected / 1 selected                                                      

unzipped_files/starter_kit/src/train.py::test_optimize PASSED                                [100%]

================================= 1 passed, 3 deselected in 13.30s =================================


Finally, complete the `test` function then run the tests:

In [15]:
!pytest -vv unzipped_files/starter_kit/src/train.py -k one_epoch_test

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 4 items / 3 deselected / 1 selected                                                      

unzipped_files/starter_kit/src/train.py::test_one_epoch_test PASSED                          [100%]

================================= 1 passed, 3 deselected in 38.52s =================================


---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 5: Putting everything together

Allright, good job getting here! Now it's time to see if all our hard work pays off. In the following cell we will train your model and validate it against the validation set.

Let's start by defining a few hyperparameters. Feel free to experiment with different values and try to optimize your model:

In [16]:
batch_size = 64
valid_size = 0.2
num_epochs = 119
num_classes = 50
dropout = 0.4
learning_rate = 0.0005
weight_decay = 0.0005

Validating: 100%|███████████████████████████████| 16/16 [00:10<00:00,  1.50it/s]

Epoch: 106 	Training Loss: 2.095909 	Validation Loss: 2.166550 	Training Accuracy: 42.92% 	Validation Accuracy: 45.10%
Early stopping triggered.


In [17]:
from unzipped_files.starter_kit.src.data import get_data_loaders
from unzipped_files.starter_kit.src.train import optimize
from unzipped_files.starter_kit.src.optimization import get_optimizer, get_loss
from unzipped_files.starter_kit.src.model import MyModel

data_loaders = get_data_loaders(batch_size=batch_size, valid_size=valid_size)

model = MyModel(num_classes=num_classes, dropout=dropout)

optimizer = get_optimizer(model, optimizer="adamw", learning_rate=learning_rate, weight_decay=weight_decay)

loss = get_loss()

optimize(
    data_loaders,
    model,
    optimizer,
    loss,
    n_epochs=num_epochs,
    save_path="checkpoints/best_val_loss.pt",
    interactive_tracking=True
)

---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 6: testing against the Test Set

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> only run this *after* you have completed hyperpameter optimization. Do not optimize hyperparameters by looking at the results on the test set, or you might overfit on the test set (bad, bad, bad)

Run the code cell below to try out your model on the test dataset of landmark images. Ensure that your test accuracy is greater than 50%.

In [18]:
# load the model that got the best validation accuracy
from unzipped_files.starter_kit.src.train import one_epoch_test
from unzipped_files.starter_kit.src.model import MyModel
import torch

model = MyModel(num_classes=num_classes, dropout=dropout)

# load the weights in 'checkpoints/best_val_loss.pt'
model.load_state_dict(torch.load('checkpoints/best_val_loss.pt'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run test
one_epoch_test(data_loaders['test'], model, loss, device)

Testing: 100%|██████████████████████████████████| 20/20 [00:12<00:00,  1.54it/s]

Test Loss: 1.877143


Test Accuracy: 50% (629/1250)


(1.8771425038576126, 50.32)

---
## <img src="static_images/icons/noun-advance-2109145.png" alt=">" style="width:50px"/> Step 7: Export using torchscript

Great job creating your CNN models! Now that you have put in all the hard work of creating accurate classifiers, let's export it so we can use it in our app.

But first, as usual, we need to complete some code!

Open `src/predictor.py` and fill up the missing code, then run the tests:

In [19]:
!pytest -vv unzipped_files/starter_kit/src/predictor.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1, typeguard-4.3.0
collected 1 item                                                                                   

unzipped_files/starter_kit/src/predictor.py::test_model_construction PASSED                  [100%]

======================================== 1 passed in 3.70s =========================================


Allright, now we are ready to export our model using our Predictor class:

In [20]:
# NOTE: you might need to restart the notebook before running this step
# If you get an error about RuntimeError: Can't redefine method: forward on class
# restart your notebook then execute only this cell
from unzipped_files.starter_kit.src.predictor import Predictor
from unzipped_files.starter_kit.src.helpers import compute_mean_and_std
from unzipped_files.starter_kit.src.model import MyModel
from unzipped_files.starter_kit.src.data import get_data_loaders
import torch

data_loaders = get_data_loaders(batch_size=1)

# First let's get the class names from our data loaders
class_names = data_loaders["train"].dataset.classes

# Then let's move the model_transfer to the CPU
# (we don't need GPU for inference)
model = MyModel(num_classes=50, dropout=0.5).cpu()

# Let's make sure we use the right weights by loading the
# best weights we have found during training
# NOTE: remember to use map_location='cpu' so the weights
# are loaded on the CPU (and not the GPU)

model.load_state_dict(torch.load('checkpoints/best_val_loss.pt', map_location='cpu'))

# Let's wrap our model using the predictor class
mean, std = compute_mean_and_std()
predictor = Predictor(model, class_names, mean, std).cpu()

# Export using torch.jit.script
scripted_predictor = torch.jit.script(predictor)

scripted_predictor.save("checkpoints/original_exported.pt")

Reusing cached mean and std
Dataset mean: tensor([0.4638, 0.4725, 0.4687]), std: tensor([0.2697, 0.2706, 0.3017])
Reusing cached mean and std


Now let's make sure the exported model has the same performance as the original one, by reloading it and testing it. The Predictor class takes different inputs than the non-wrapped model, so we have to use a specific test loop:

In [21]:
import torch

# Load using torch.jit.load
model_reloaded =  torch.jit.load('checkpoints/original_exported.pt')

In [22]:
from unzipped_files.starter_kit.src.predictor import predictor_test

pred, truth = predictor_test(data_loaders['test'], model_reloaded)

100%|███████████████████████████████████████| 1250/1250 [08:07<00:00,  2.57it/s]

Accuracy: 0.5016


Finally, let's have a look at the confusion matrix of the model we are going to use in production:

In [23]:
from unzipped_files.starter_kit.src.helpers import plot_confusion_matrix

plot_confusion_matrix(pred, truth)

In [25]:
import shutil
from google.colab import files

shutil.make_archive('checkpoints', 'zip', '/content/checkpoints')
files.download('/content/checkpoints.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>